In [134]:
import pandas as pd
import os
import glob

'''Convert Between Dire and Radiant'''
def opposite_team(team):
    if team == 'Dire':
        response = 'Radiant'
    else:
        response = 'Dire'
    return response

all_matches = glob.glob(os.path.join('data/matches_upload/', "*.csv")) 
# In one line concatenate and drop duplicates of all matches files
match_uploads = pd.concat(pd.read_csv(f, index_col=False) for f in all_matches).drop_duplicates()
match_uploads.rename(columns={
    '﻿"Match ID"': 'MatchID',
    'Start Date/Time': 'StartTime',
    'Match Length': 'MatchLength',
    'Duration (secs)': 'Duration',
    'Kills Score': 'KillScore',
    'Unnamed: 7': 'KillScore2',
    'Radiant Team': 'Radiant',
    'Radiant Players': 'RadiantPlayers',
    'Dire Team': 'Dire',
    'Dire Players': 'DirePlayers'}, inplace=True)
match_uploads['RadiantKills'] = match_uploads.KillScore.str.split('-').apply(lambda x:x[0])
match_uploads['DireKills'] = match_uploads.KillScore.str.split('-').apply(lambda x:x[1])
match_uploads.drop(['KillScore', 'KillScore2'], 1, inplace=True)
match_uploads['WinningTeam'] = match_uploads.apply(lambda x: x[x.Winner], axis=1)
match_uploads['LosingTeam'] = match_uploads.apply(lambda x: x[opposite_team(x.Winner)], axis=1)
match_uploads.to_csv('data/matches.csv', index=False)

all_teams = glob.glob(os.path.join('data/team_upload/', "*.csv"))
team_uploads = pd.DataFrame()
for files in all_teams:
    data = pd.read_csv(files) 
    # add upload date from filename by splitting around parenthesis
    data['UploadDate'] = files.split('(', 1)[1].split(')', 1)[0]
    team_uploads = pd.concat([team_uploads, data])  # Concatenate the DataFrames 
team_uploads.rename(columns={
    'Unnamed: 1': 'TeamName',
    'Total Count': 'TotalCount',
    'As Radiant': 'RadiantGames',
    'As Dire': 'DireGames',
    'Kills': 'AvgKills',
    'Deaths': 'AvgDeaths',
    'Assists': 'AvgAssists',
    'GPM': 'AvgGPM',
    'XPM': 'AvgXPM',
    'Last Hits': 'AvgLastHits',
    'Denies': 'AvgDenies',
    'Overall': 'AvgTime',
    'In Wins': 'AvgWinTime',
    'In Losses': 'AvgLossTime'}, inplace=True)
team_uploads['WinRate'] = team_uploads.Winrate.str.split('%').apply(lambda x: x[0])
team_uploads['WinRate'] = pd.to_numeric(team_uploads.WinRate) / 100
team_uploads.drop(['Winrate'], 1, inplace=True)
team_uploads.to_csv('data/teams.csv', index=False)